In [ ]:
# Execute this command to check GPU model
# In Colab the best one is Tesla P100
!nvidia-smi

In [ ]:
# Execute this to mount drive (if needed)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/DeepNILM')
import data_loading
import data_understanding
import data_model
import model_training
import model_testing
import data_ingestion
import metrics

In [ ]:
# To Reload modules after updating them
import importlib
importlib.reload(data_loading)
importlib.reload(data_understanding)
importlib.reload(data_model)
importlib.reload(model_training)
importlib.reload(model_testing)
importlib.reload(data_ingestion)
importlib.reload(metrics)

The purpose of this notebook is to train a deep neural network to solve the following Energy Disaggregation task: infer the power consumption of two appliances (dishwasher and fridge) given the overall consumption (main) of an house.

The followed approach is mainly inspired by the Sequence-to-point one described by Zhang et al. (2017) (https://arxiv.org/abs/1612.09106).

## Data Loading

In [ ]:
dishwasher_power = data_loading.read_csv_data("drive/My Drive/Progetto/dishwasher_train.csv")
fridge_power = data_loading.read_csv_data('drive/My Drive/Progetto/fridge_train.csv')
main_power = data_loading.read_csv_data('drive/My Drive/Progetto/main_train.csv')

## Data Understanding

We make first an explorative analysis of the time series.

In [ ]:
print('###################################################')
print('Data Understanding for dishwasher power consumption')
print('###################################################')
data_understanding.data_explore(dishwasher_power)
print('###################################################')
print('Data Understanding for fridge power consumption')
print('###################################################')
data_understanding.data_explore(fridge_power)
print('###################################################')
print('Data Understanding for main power consumption')
print('###################################################')
data_understanding.data_explore(main_power)

Then, we visualize the time series in a given day.

In [ ]:
start_time_visualization = '2019-02-06 00:00:00'
end_time_visualization = '2019-02-07 00:00:00'
data_understanding.plot_interval(main_power, start_time_visualization,
                                 end_time_visualization, title='Random Daily Overall Power Consumption')
data_understanding.plot_interval(dishwasher_power, start_time_visualization,
                                 end_time_visualization, title='Random Daily Dishwasher Power Consumption')
data_understanding.plot_interval(fridge_power, start_time_visualization,
                                 end_time_visualization, title='Fridge Daily Power Consumption')

## Dishwasher's consumption inference

Let's train the model for dishwasher with early stopping enabled.

In [ ]:
model_training.train_model(appliance_name='dishwasher', 
                           main_path='drive/My Drive/Progetto/main_train.csv',
                           appliance_path='drive/My Drive/Progetto/dishwasher_train.csv',
                           train_end_timestamp='2019-03-01 00:00:00',
                           window_size=600,
                           batch_size=512,
                           build_model_func=data_model.build_dishwasher_cnn,
                           epochs=20,
                           patience=4,
                           early_stopping=True,
                           rescaling='normalize',
                           split=True,
                           plot_model=True)

Now we train the model for a fixed number of epochs.

In [ ]:
model = model_training.train_model(appliance_name='dishwasher',
                                   main_path='drive/My Drive/Progetto/main_train.csv',
                                   appliance_path='drive/My Drive/Progetto/dishwasher_train.csv',
                                   window_size=600,
                                   batch_size=512,
                                   build_model_func=data_model.build_dishwasher_cnn,
                                   epochs=8,
                                   rescaling='normalize')

The model is saved for future reuse.

In [ ]:
model.save('drive/My Drive/DeepNILM/dishwasher_model')

Let's try the test module.

In [ ]:
ground_truth, predicted_values = model_testing.test_model(appliance_name='dishwasher',
                                                          main_path='drive/My Drive/Progetto/main_train.csv',
                                                          appliance_path='drive/My Drive/Progetto/dishwasher_train.csv',
                                                          model_path='drive/My Drive/DeepNILM/dishwasher_model',
                                                          window_size=600,
                                                          batch_size=512,
                                                          rescaling='normalize',
                                                          appliance_min_power=0.0,
                                                          appliance_max_power=2570.6,
                                                          main_min_power=73.48100000000002,
                                                          main_max_power=6048.699999999999)

In [ ]:
f1 = metrics.compute_F1_score(predicted_values, ground_truth)
print('Energy based F1 score on test set: {}'.format(f1))

## Fridge's Consumption inference

Let's train the model for fridge with early stopping enabled.



In [ ]:
model_training.train_model(appliance_name='fridge', 
                           main_path='drive/My Drive/Progetto/main_train.csv',
                           appliance_path='drive/My Drive/Progetto/fridge_train.csv',
                           train_end_timestamp='2019-03-01 00:00:00',
                           window_size=600,
                           batch_size=512,
                           build_model_func=data_model.build_fridge_cnn,
                           epochs=50,
                           patience=10,
                           early_stopping=True,
                           rescaling='standardize',
                           split=True,
                           plot_model=True)

Now we train the model for a fixed number of epochs.

In [ ]:
model = model_training.train_model(appliance_name='fridge',
                                   main_path='drive/My Drive/Progetto/main_train.csv',
                                   appliance_path='drive/My Drive/Progetto/fridge_train.csv',
                                   window_size=600,
                                   batch_size=512,
                                   build_model_func=data_model.build_fridge_cnn,
                                   epochs=7,
                                   rescaling='standardize')

The model is saved for future reuse.

In [ ]:
model.save('drive/My Drive/DeepNILM/fridge_model')

Let's try the test module.

In [ ]:
ground_truth, predicted_values = model_testing.test_model(appliance_name='fridge',
                                                          main_path='drive/My Drive/Progetto/main_train.csv',
                                                          appliance_path='drive/My Drive/Progetto/fridge_train.csv',
                                                          model_path='drive/My Drive/DeepNILM/fridge_model',
                                                          window_size=600,
                                                          batch_size=512,
                                                          rescaling='standardize',
                                                          appliance_mean_power=37.23710644724372,
                                                          appliance_std_power=46.9886959530205,
                                                          main_mean_power=370.91555422946004,
                                                          main_std_power=549.1880538356259)

In [ ]:
f1 = metrics.compute_F1_score(predicted_values, ground_truth)
print('Energy based F1 score on test set: {}'.format(f1))